## layers/Estimater との連携がうまく行っていないので調査が必要

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from google.datalab.ml._tensorboard import TensorBoard
import matplotlib.pyplot as plt

In [5]:
# MNISTのデータのダウンロードと展開
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [21]:
def model_fn(x, y, mode, params, config, model_dir):
  x_image = tf.reshape(x, [-1, 28, 28, 1])
  conv1 = tf.layers.conv2d(x_image, 8, (5, 5), padding='same', activation=tf.nn.relu)
  pool1 = tf.layers.max_pooling2d(conv1, 2, 2)
  conv2 = tf.layers.conv2d(pool1, 16, (5, 5), padding='same', activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(conv2, 2, 2)

  fc = tf.contrib.layers.flatten(pool2)
  for i in range(params.get('n_fc_layers', 1)):
    fc = tf.layers.dense(fc, 128, activation=tf.nn.relu)
  logits = tf.layers.dense(fc, 10, activation=tf.nn.relu)
  
  xent = tf.losses.softmax_cross_entropy(y, logits)
  tf.summary.scalar('xent', xent)
   
  train_op = tf.contrib.layers.optimize_loss(
    loss=xent,
    global_step=tf.contrib.framework.get_global_step(),
    learning_rate=params.get('learning_rate', 1e-4),
    optimizer=tf.train.AdamOptimizer
  )
    
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('acurracy', accuracy)
  
  return tf.contrib.learn.ModelFnOps(
    mode=mode,
    predictions={
      'labels': tf.argmax(logits, 1),
      'softmax': tf.nn.softmax(logits)
    },
    loss=xent,
    train_op=train_op
  )

In [22]:
nn = tf.contrib.learn.Estimator(
  model_fn=model_fn,
  params={'learning_rate': 1e-4, 'n_fc_layers': 1},
  model_dir='./logs/layers/4'
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fedf8aa2f90>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [ ]:
#summary_writer = tf.summary.FileWriter('logs/layers/2')

nn.fit(
  mnist.train.images,
  mnist.train.labels,
  #monitors=[tf.contrib.learn.monitors.SummarySaver(tf.summary.merge_all(), save_steps=5, summary_writer=summary_writer)]
)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


/usr/local/lib/python2.7/dist-packages/tensorflow/python/util/deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./logs/layers/4/model.ckpt.
INFO:tensorflow:loss = 2.30395, step = 1
